In [1]:
import netpixi
from graph_tool import draw
import graph_tool_extras as gte

import time

from data import PRODUCTS

In [6]:
def get_or_add_vertex(g, id):
    v = g.vertex_by_id(id)
    if v is None:
        v = g.add_vertex_by_id(id)
        p = PRODUCTS[str(id)]
        v['aisle'] = p['aisle']
        v['department'] = v['department']
        v['name'] = p['name']
    return v

def get_or_add_edge(g, a, b, v):
    e = g.edge_by_ids(a, b)
    if e is None:
        e = g.add_edge_by_ids(a, b)
        e['orders'] = v
    return e

In [7]:
g = gte.Graph(directed=False)
g.add_ep('orders')
g.add_vp('aisle')
g.add_vp('department')
g.add_vp('name')

with open('./data/edges.csv') as file:
    for line in file:
        a, b, v = [int(x) for x in line.strip().replace('\n', '').split(',')]
        vA = get_or_add_vertex(g, a)
        vB = get_or_add_vertex(g, b)
    
        e = get_or_add_edge(g, a, b, v)

### 04: Construindo e renderizando a Rede

In [8]:
layout = draw.sfdp_layout(g)

In [9]:
gte.move(g, layout)

In [10]:
g = gte.clean(g)

In [11]:
gte.save(g, 'rede.net.gz')

In [ ]:
r = netpixi.render('./rede.net.gz', infinite=True)

In [9]:
time.sleep(5)
r.vertex_default(size=4, bwidth=1)
r.edge_default(width=1)